In [1]:
import pandas as pd
import numpy as np
import json
import google.generativeai as genai
from google.generativeai.types import HarmBlockThreshold, HarmCategory
import json

with open('/Users/lekshmanannatarajan/Library/CloudStorage/GoogleDrive-zuari1993@gmail.com/My Drive/Colab Notebooks/config.txt', 'r') as config_file:
    config = json.load(config_file)
API_KEY = config["API_KEY"]
genai.configure(api_key=API_KEY)

synthetic_train_data = pd.DataFrame(columns = ['Sn.no', 'user_prompt', 'truth_lable'])
synthetic_train_data

/Users/lekshmanannatarajan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Sn.no,user_prompt,truth_lable


In [2]:
GEMINI_MODELS = {
    'FLASH':"gemini-1.5-flash-002"
    ,'PRO':"gemini-1.5-pro-002"
    ,'FLASH_8B':"gemini-1.5-flash-8b-001"
}

model_safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

generation_config = genai.GenerationConfig(
    max_output_tokens=3000,
    temperature=0.8,
    top_k=1,
    top_p=0.95,
)

In [3]:
# model = genai.GenerativeModel(GEMINI_MODELS['FLASH']
#                             , system_instruction= "create varied, human-like responses, avoiding repetition"
#                                 , generation_config = generation_config
#                                 , safety_settings = model_safety_settings)

In [5]:
# response = model.generate_content(contents = ["""
# I need training/validation data to fine tune a LLM classifier I am building. It will classify user prompts to either ['help me brainstorm', 'summarize this report for me']. Generate 5 prompts for each of the mentioned categories.
# """])

# print(response.text)

In [11]:
# Define the configuration with structured output
generation_config = genai.GenerationConfig(
    max_output_tokens=1000,
    temperature=0.7,
    top_k=1,
    top_p=0.95,
    response_mime_type="application/json",
    response_schema={
        "type": "array",
        "items": {
            "type": "object",
            "properties": {
                "prompt": {"type": "string", "description": "Generated user prompt"},
                "truth_label": {"type": "string", "enum": ["help me brainstorm", "summarize this report for me"]}
            },
            "required": ["prompt", "truth_label"]
        }
    }
)

# Create the model
gemini_model = genai.GenerativeModel(
    model_name=GEMINI_MODELS['FLASH'],
    system_instruction=(
        "You are an AI agent trained to generate structured output."
        "for each category in ['help me brainstorm', 'summarize this report for me']."
        "Ensure the output adheres to the defined JSON schema."
    ),
    generation_config=generation_config,
    safety_settings=model_safety_settings
)

# Generate structured content
response = gemini_model.generate_content(
    contents=["""I need training/validation data to fine-tune a LLM classifier. Generate 10 prompts for each category."""])
    
# Print the JSON output
print(response.text)

[{"prompt": "Suggest creative marketing campaign ideas for a new sustainable fashion brand.", "truth_label": "help me brainstorm"}, {"prompt": "I need innovative ideas for a mobile game that incorporates augmented reality.", "truth_label": "help me brainstorm"}, {"prompt": "Help me brainstorm potential solutions to reduce traffic congestion in urban areas.", "truth_label": "help me brainstorm"}, {"prompt": "Let's brainstorm unique gift ideas for a friend who loves to travel.", "truth_label": "help me brainstorm"}, {"prompt": "Brainstorm potential storylines for a science fiction novel with a female protagonist.", "truth_label": "help me brainstorm"}, {"prompt": "I need ideas for a fundraising event to support a local animal shelter.", "truth_label": "help me brainstorm"}, {"prompt": "Help me brainstorm ways to improve employee engagement and morale at my company.", "truth_label": "help me brainstorm"}, {"prompt": "Let's brainstorm potential names for a new coffee shop in a historic dis

In [12]:
pd.DataFrame(json.loads(response.text))

,prompt,truth_label
0,Suggest creative marketing campaign ideas for ...,help me brainstorm
1,I need innovative ideas for a mobile game that...,help me brainstorm
2,Help me brainstorm potential solutions to redu...,help me brainstorm
3,Let's brainstorm unique gift ideas for a frien...,help me brainstorm
4,Brainstorm potential storylines for a science ...,help me brainstorm
5,I need ideas for a fundraising event to suppor...,help me brainstorm
6,Help me brainstorm ways to improve employee en...,help me brainstorm
7,Let's brainstorm potential names for a new cof...,help me brainstorm
8,Brainstorm creative solutions for reducing pla...,help me brainstorm
9,I need ideas for a team-building activity that...,help me brainstorm
